In [1]:
import time
import re
import pickle
import pandas as pd
import numpy as np

In [7]:
with open("srez.pickle", "rb") as f:
    rsrez = pd.DataFrame(pickle.load(f))

In [9]:
rsrez = rsrez[rsrez["Часть речи"].isin(["ПРИЛ", "НАР", "СУЩ", "", "ГЛАГ", "МЕСТ", "ПРЕД"]) & (~rsrez["Значения"].isin(["", "?"]))]

rsrez_pos = {
    'ПРИЛ': "adjective",
    'НАР': "adverb",
    'СУЩ': "noun",
    'ГЛАГ': "verb",
    'МЕСТ': "pronoun",
    '': None,
    'ПРЕД': "preposition"
}

rsrez = rsrez.assign(
    word=rsrez["Словарная статья"].str.lower(),
    definition=rsrez["Значения"].map(lambda x: re.sub(r"\n", " ", x)).map(lambda x: re.sub(r"[^\w\ ]", "", x).strip()),
    pos=srez["Часть речи"].map(srez_pos),
    sourse="sreznevskiy"
)[["old_rus_word", "definition", "pos", "sourse"]]

In [166]:
rsrez.head()

,word,translation,pos,sourse
5,абидныи,обидный,adjective,sreznevskiy
6,абиѥ,тот час,adverb,sreznevskiy
7,аблань,яблоня,noun,sreznevskiy
8,абланьныи,яблоневый,adjective,sreznevskiy
9,аблъко,яблоко,noun,sreznevskiy


In [171]:
or_dict = rsrez[rsrez.definition.map(lambda x: " " not in x)]

In [172]:
or_w2w = rsrez[rsrez.definition.map(lambda x: " " in x)]

In [66]:
with open("parsed_410_tomes.pickle", "rb") as f:
    t410 = pickle.load(f)

In [82]:
t410 = t410[~t410.definition.astype(str).str.contains(r"^(?:к|то)\ ", case=False)]

In [98]:
t410 = t410[~t410.word.str.lower().isin(rsrez.old_rus_word.values)]

t410 = t410[~t410["info"].map(lambda x: "союз." in x if (x == x) else False)]

t410 = t410[t410.definition.notna()]

In [140]:
def ps_conv(l):
    if l != l:
        return None
    conv_dict = {
        "с.": "noun",
        "пр.": "adjective",
        "нар.": "adverb",
        "гл.": "verb",
        "мест.": "pronoun"
    } 
    for k, v in conv_dict.items():
        if k in l:
            return v
    return None

def clean_t410_definition(inp_s):
    s = inp_s.lower()
    s = re.sub(r"[^\w\ ]", "", s)
    s = " ".join([x for x in s.split() if all(y in "абвгдеёжзийклмнопрстуфхцчшщъыьэюя" for y in x)])
    return s

In [146]:
t410 = t410.assign(
    word=t410["word"].str.lower(),
    definition=t410["definition"].map(clean_t410_definition),
    pos=t410["info"].map(ps_conv),
    sourse="avanesov"
)[["word", "definition", "pos", "sourse"]]

In [284]:
# t410.head()

In [295]:
or_dict = or_dict.append(or_w2w, ignore_index=True)

In [181]:
or_dict = or_dict.append(t410[t410.translation.map(lambda x: " " not in x)], ignore_index=True)
or_dict = or_dict.assign(word=or_dict.word.str.replace(r"c", "с").map(lambda x: re.sub(r"(?:ѩ|˫а)", "я", x)))

In [297]:
leters = set()
for x in or_dict.word.values:
    leters.update(set(x))


In [300]:
# with open("or_dict.pickle", "wb") as f:
#     pickle.dump(or_dict, f)

In [2]:
with open("or_dict.pickle", "rb") as f:
    or_dict = pickle.load(f)

In [4]:
len(or_dict)

39329

In [ ]:
with open("or_dict.pickle", "rb") as f:
    or_dict = pickle.load(f)

In [25]:
or_dict.pos.unique()

array(['adjective', 'noun', 'adverb', 'pronoun', 'verb', None,
       'preposition'], dtype=object)

In [8]:
with open("oed.pickle", "rb") as f:
    oed = pickle.load(f)

In [15]:
oed.loc[11002]

definition    a payment
pos                 sb.
word              yelde
Name: 11002, dtype: object

In [23]:
oed = oed[oed.word.map(lambda x: len(x) > 2)]

In [ ]:
['adjective', 'noun', 'adverb', 'pronoun', 'verb', None,
       'preposition']

In [ ]:
sb
adj
adv
prep pp
pron
v.


In [32]:
def pos_oed(s):
    if s == s:
        pos = {
            "sb.": "noun",
            "adj.": "adjective",
            "adv.": "adverb",
            "prep.": "preposition",
            "pp.": "preposition",
            "pron.": "pronoun",
            "v.": "verb",
        }
        for k in pos:
            if k in s:
                return pos[k]

In [34]:
oed = oed.assign(pos=oed.pos.map(pos_oed))

In [47]:
reg = re.compile(r"\b(?:t(?:he|o)|a)\ ")
oed = oed.assign(definition=oed.definition.map(lambda x: reg.sub("", x)))

In [53]:
oe_dict = oed.assign(source="mayhew_skeat")

In [54]:
# with open("oe_dict.pickle", "wb") as f:
#     pickle.dump(oe_dict, f)

In [ ]:
with open("oec.pickle", "rb") as f:
    oec = pickle.load(f)

In [ ]:
with open("doe.pickle", "rb") as f:
    doe = pickle.load(f)